<a href="https://colab.research.google.com/github/Tatsuro0726/Scraping/blob/master/Chapter7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Chapter7-1: 類似画像の検出を使用
#### 簡単な形状認識 - Average Hash
- 画像を比較可能なハッシュ値で表す。ハッシュ関数:MD5, SHA256などを使う。⇒ データの値を要約したハッシュ値を得られる。
- 画像データは一般的に同一ではない(サイズとか色調, jpeg/pngなど)のため、ハッシュ関数を使えない。
- 類似度計算に使用するAverage Hashが役立つ
  1. 画像のサイズを8×8
  1. 色をグレースケールに変換
  1. 画像の各ピクセルの平均値を計算
  1. 各ピクセルの濃淡が平均より大きければ1, 平均以下なら0とする
- 上記の手順で、8×8=64bitのハッシュ値が得られる。⇒ 64ビットを比較するだけで済む。

In [ ]:
# Pythonの画像ライブラリーPillowをインストール
!pip install Pillow

In [ ]:
# avghash
from PIL import Image
import numpy as np

# 画像データをAverage Hashに変換
def average_hash(fname, size = 16):
  img = Image.open(fname)
  img = img.convert('L') # グレースケールに変換
  img = img.resize((size, size), Image.ANTIALIAS) # リサイズ　アンチエイリアス：境界線を滑らかに見せる
  pixel_data = img.getdata() # ピクセルデータを得る
  pixels = np.array(pixel_data) # numpy配列に変換
  pixels = pixels.reshape((size, size)) # 2次元配列に変換
  avg = pixels.mean() # 算術平均を計算
  diff = 1 * (pixels > avg) # 平均より大きければ1, 平均以下で0に変換
  return diff

# 2進数とみなしてハッシュ値に変換
def np2hash(n):
  bhash = []
  for nl in ahash.tolist():
    sl = [str(i) for i in nl]
    s2 = ''.join(sl)
    i = int(s2,2) # 2進数を整数に
    bhash.append('%04x' % i)
  return ''.join(bhash)

# Average Hashを表示
ahash = average_hash('./skytree.jpg')
print(ahash)
print(np2hash(ahash))

[[0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0]
 [0 0 1 1 1 1 0 1 1 1 0 1 0 0 0 0]
 [1 1 1 1 1 0 0 1 1 1 1 1 0 1 1 1]
 [1 1 1 1 1 0 0 0 1 1 1 1 0 0 1 1]
 [1 1 1 1 1 0 0 0 1 1 1 1 0 0 1 1]
 [1 1 1 1 1 0 0 0 1 1 1 1 0 0 1 1]
 [1 1 1 1 1 0 0 0 1 1 1 1 1 0 1 1]
 [1 1 1 1 0 0 0 0 1 1 1 1 1 1 1 1]
 [1 1 1 1 0 0 0 0 1 1 1 1 0 1 1 1]
 [1 1 1 1 0 0 0 0 1 1 1 1 0 1 1 1]
 [1 1 1 0 0 0 0 0 1 1 1 1 1 0 1 1]
 [1 1 1 0 0 0 0 0 0 1 1 1 1 0 1 1]
 [1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 1]
 [1 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1]
 [1 1 0 0 0 0 0 0 0 1 1 1 0 0 1 1]
 [1 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1]]
1f803dd0f9f7f8f3f8f3f8f3f8fbf0fff0f7f0f7e0fbe07be07fc07fc073807b


In [4]:
from PIL import Image
import numpy as np
import os, re

# ファイルパスの指定
search_dir = './drive/My Drive/Colab Folder/images/flowers'
cache_dir = './drive/My Drive/Colab Folder/images/cache_avhash'

# Cacheフォルダ作成
if not os.path.exists(cache_dir):
  os.mkdir(cache_dir)

# 画像データをAverage hashに変換
def average_hash(fname, size=16):
  fname2 = fname[len(search_dir):]
  # 画像をキャッシュしておく
  cache_file = cache_dir + '/' + fname2.replace('/', '_') + 'csv'
  if not os.path.exists(cache_file): # ハッシュを作成
    img = Image.open(fname)
    img = img.convert('L').resize((size, size), Image.ANTIALIAS)
    pixels = np.array(img.getdata()).reshape((size, size))
    avg = pixels.mean()
    px = 1 * (pixels > avg)
    np.savetxt(cache_file, px, fmt='%.0f', delimiter=',')
  else: # すでにキャッシュがあればファイルから読み込み
    px = np.loadtxt(cache_file, delimiter=',')
  return px

# 簡単にハミング距離を求める
def hamming_dist(a, b):
  aa = a.reshape(1,-1) # 1次元配列に変換
  ab = b.reshape(1,-1)
  dist = (aa != ab).sum()
  return dist

# すべてのディレクトリを列挙
def enum_all_files(path):
  for root, dirs, files in os.walk(path):
    for f in files:
      fname = os.path.join(root, f)
      if re.search(r'\.(jpg|jpeg|png)$',fname):
        yield fname

# 画像を検索
def find_image(fname, rate):
  src = average_hash(fname)
  for fname in enum_all_files(search_dir):
    dst = average_hash(fname)
    diff_r = hamming_dist(src, dst) / 256
    # print('[check]', fname)
    if diff_r < rate:
      yield (diff_r, fname)

# 検索
srcfile = search_dir + '/rose/mountain_rose_s_000071.png'
html = ''
sim = list(find_image(srcfile, 0.25))
sim = sorted(sim, key=lambda x: x[0])
for r, f in sim:
  print(r, '>', f)
  s = '<div style="float:left;"><h3>[差異:' + str(r) + '-' + \
    os.path.basename(f) + ']</h3>'+ \
    '<p><a href="' + f + '"><img src="' + f + '" width=400>'+ \
    '</a></p></div>'
  html += s

# html出力
html = """<html><body><h3>元画像</h3><p>
<img src='{0} width=400></p>{1}
</body></html>""".format(srcfile, html)
with open("./avhash-search-output.html", "w", encoding="utf-8") as f:
  f.write(html)
print('ok')

0.0 > ./drive/My Drive/Colab Folder/images/flowers/rose/mountain_rose_s_000071.png
0.2421875 > ./drive/My Drive/Colab Folder/images/flowers/tulip/tulip_s_000227.png
ok


### Chapter7-2: CNNで画像分類に挑戦しよう
CNNでカラー画像の分類に挑戦する

In [5]:
# 画像データをPythonのデータ型に変換する
from sklearn.model_selection import cross_validate
from PIL import Image
import os, glob
import numpy as np

# 分類対象のカテゴリーを選ぶ
image_dir = './drive/My Drive/Colab Folder/images/household_furniture' 

ImportError: ignored